In [10]:
import os
import pandas as pd
from labels import LABELS

os.environ["TOKENIZERS_PARALLELISM"] = "false"

data = "./dataset_resume.csv"
df = pd.read_csv(data)

df_new = df.copy()

for index, row in df_new.iterrows():
    df_new.loc[index, "description"] = (
            row.resume
            if isinstance(row.resume, str)
            else row.description
        )

In [11]:
from sklearn.model_selection import train_test_split

def setup(df):
    train_df, test_data = train_test_split(
        df, test_size=0.2, random_state=42, shuffle=True
    )

    val_df, test_df = train_test_split(test_data, test_size=0.5)

    return train_df, test_df, val_df

In [12]:
from datasets import Dataset

train_df, test_df, val_df = setup(df_new)

train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
val_ds = Dataset.from_pandas(val_df)

In [13]:
from setfit import SetFitModel, Trainer, TrainingArguments



model = SetFitModel.from_pretrained("camembert-base")

args = TrainingArguments(
    batch_size=3,
    num_epochs=1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    column_mapping=['']
)
trainer.train()

No sentence-transformers model found with name camembert-base. Creating a new one with mean pooling.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


ValueError: SetFit expected the dataset to have the columns ['label', 'text'], but only the columns ['Action', 'Art', 'Atelier', 'Balade', 'Brocante', 'Concert', 'Conférence', 'Culture', 'Danse', 'Détente', 'Environnement', 'Exposition', 'Famille', 'Festival', 'Fête', 'Gastronomie', 'Histoire', 'Jeu', 'Marché', 'Santé', 'Spectacle', 'Sport', 'Théatre', 'Unnamed: 0', 'Visite', '__index_level_0__', 'description', 'resume', 'title'] were found. Either make sure these columns are present, or specify which columns to use with column_mapping in Trainer.